In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import pickle
import string
import json
import random
import csv
from collections import Counter
#MUSE Part
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import norm
import matplotlib.mlab as mlab
from sklearn.neighbors import NearestNeighbors
import copy

import sys
import codecs

In [104]:
src = "pop"

In [118]:
trg = "metal"

In [61]:
filepath = "data/lyrics_final_clean.csv"
lyrics_df = pd.read_csv(filepath)
lyrics_df = lyrics_df.dropna(axis = 0, how='any', subset=['lyrics'])

In [62]:
def load_embeddings(file_name):
    with codecs.open(file_name, 'r', 'utf-8') as f_in:
        lines = f_in.readlines()
        lines = lines[1:]
        vocabulary, wv = zip(*[line.strip().split(' ', 1) for line in lines])
    wv = np.loadtxt(wv)
    return wv, vocabulary

In [119]:
word_embeddings_src, vocabulary_src = load_embeddings('data/MUSE/model_lyrics_'+src+'.vec')
word_embeddings_trg, vocabulary_trg = load_embeddings('data/MUSE/model_lyrics_'+trg+'.vec')

In [120]:
voc_src = list(vocabulary_src)
voc_trg = list(vocabulary_trg)

In [121]:
common_words = set(voc_src).intersection(set(voc_trg))
common_words = list(common_words)

In [122]:
src_specific_from_trg = set(voc_src).difference(set(common_words))

In [90]:
pickle.dump(src_specific_from_trg, open("webpage/"+src+"/specific_from_"+trg+".py", "wb"))

In [123]:
corpus_src = list(lyrics_df[lyrics_df['genre'] == src]['lyrics'])
corpus_src = [x.replace('\n', ' ') for x in corpus_src]

In [70]:
def get_tfidf(corpus, max_freq, min_freq):
    #We use the stop_words parameter to remove the current words from the computations
    tf = TfidfVectorizer(stop_words='english', max_df=max_freq, min_df=min_freq)
    X = tf.fit(corpus)
    X_transformed = X.transform(corpus)
    #Find maximum value for each of the features over all of dataset
    max_val = X_transformed.max(axis=0).toarray().ravel()

    feature_names = np.array(tf.get_feature_names())
    sorted_by_idf = np.argsort(tf.idf_)
    #sort weights from smallest to biggest and extract their indices 
    sort_by_tfidf = max_val.argsort()
    return feature_names[sort_by_tfidf]

In [124]:
tfidf_src = get_tfidf(corpus_src, 1.0, 0.0)

/Users/emma/anaconda3/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [93]:
pickle.dump(tfidf_src, open("webpage/"+src+"/tfidf.py", "wb"))

In [125]:
N_EPOCH = 50
BATCH_SIZE = 32
N_ITERATION = round(len(voc_src)/BATCH_SIZE)
REFINEMENT = 50 #Refinement is for the Procrustes Iterations, involved in the rotation of the embeddings
INPUT_GENRE = src
OUTPUT_GENRE = trg
MODEL_ROCK = 'data/MUSE/model_lyrics_'+ INPUT_GENRE +'.vec'
MODEL_POP = 'data/MUSE/model_lyrics_'+ OUTPUT_GENRE+'.vec'
SRC_LANG = 'MUSE_'+ INPUT_GENRE
TGT_LANG = 'MUSE_'+ OUTPUT_GENRE

In [74]:
#We will use the following method to spare us the very long output and directly extract the result
def analyse_run(data):
    data = ' '.join(data)
    #You can change the path by analysing the logs in MUSE-master/dumped/debug
    dump = data.split('exp_path: /Users/emma/Cours/Sem_3/Lyrix/REPORT/data/MUSE/MUSE-master/dumped/debug/')[1].split(' ')[0]
    substring = data.split('* Best value for "mean_cosine-csls_knn_10-S2T-10000": ')[-1]
    iteration = substring.split('End of ')[1].split('. ')[0]
    best_mean_cosine = substring.split(' INFO')[0]
    print("For dump ", dump, " the best mean cosine was ", best_mean_cosine, " reached at ", iteration)
    return dump

In [95]:
data = ! python data/MUSE/MUSE-master/unsupervised.py --src_lang $SRC_LANG --tgt_lang $TGT_LANG --src_emb $MODEL_ROCK --tgt_emb $MODEL_POP --n_epochs $N_EPOCH --epoch_size $N_ITERATION --batch_size $BATCH_SIZE --n_refinement $REFINEMENT
dump = analyse_run(data)

For dump  vlrd6t95c2  the best mean cosine was  0.59400  reached at  refinement iteration 29


- pop -> rock : For dump  phymmrk0jd  the best mean cosine was  0.59956  reached at  refinement iteration 49
- pop -> metal : For dump  nslut4e44p  the best mean cosine was  0.60609  reached at  refinement iteration 49
- rock -> hip hop :For dump  vlrd6t95c2  the best mean cosine was  0.59400  reached at  refinement iteration 29

In [126]:
dump = "nslut4e44p"

In [127]:
src_muse_emb = "data/MUSE/MUSE-master/dumped/debug/"+ dump + "/vectors-"+ SRC_LANG + ".txt"
trg_muse_emb = "data/MUSE/MUSE-master/dumped/debug/"+ dump + "/vectors-"+ TGT_LANG + ".txt"

In [128]:
muse_emb_src, muse_voc_src = load_embeddings(src_muse_emb)
muse_emb_trg, muse_voc_trg = load_embeddings(trg_muse_emb)

In [ ]:
pickle.dump(muse_emb_trg, open("webpage/"+trg+"/muse_emb_with_"+src+".py", "wb"))

In [130]:
neigh_trg = NearestNeighbors(n_neighbors=3)
neigh_trg.fit(muse_emb_trg)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=3, p=2, radius=1.0)

In [79]:
def get_dict(embed, voc):
    voc_embeds_dict = {}
    embeds_voc_dict = {}
    for v, emb in zip(voc, embed):
        voc_embeds_dict[v] = tuple(emb)
        embeds_voc_dict[tuple(emb)] = v
    return voc_embeds_dict, embeds_voc_dict

In [100]:
muse_voc2embed_src, _ = get_dict(muse_emb_src, muse_voc_src) 
_, muse_embed2voc_trg = get_dict(muse_emb_trg, muse_voc_trg) 

In [101]:
pickle.dump(muse_voc2embed_src, open("webpage/"+src+"/muse_voc2embed_with_"+trg+".py", "wb"))
pickle.dump(muse_embed2voc_trg, open("webpage/"+trg+"/muse_embed2voc_with_"+src+".py", "wb"))